In [ ]:
!pip install pennylane torch torchvision numpy matplotlib scikit-learn

import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Subset
# Define transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load MNIST Dataset
train_data = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_data = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# # Randomly select 10,000 images from the training dataset
# subset_size = 10000
# random_indices = torch.randperm(len(train_data)).tolist()[:subset_size]  # Randomly shuffle and select 10,000 samples

# # Create a random subset of the training data
# train_data_subset = Subset(train_data, random_indices)

# # Split the subset into train and validation sets (80% train, 20% validation)
# train_size = int(0.8 * len(train_data_subset))
# val_size = len(train_data_subset) - train_size
# train_subset, val_subset = torch.utils.data.random_split(train_data_subset, [train_size, val_size])

# # Create DataLoaders for train, validation, and test
# batch_size = 128
# train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# # Show data sizes
# print(f"Training Data Size: {len(train_subset)}")
# print(f"Validation Data Size: {len(val_subset)}")
# print(f"Test Data Size: {len(test_data)}")

# Split train data into train and validation sets (80% train, 20% validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Create DataLoaders
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Show data sizes
print(f"Training Data Size: {len(train_data)}")
print(f"Validation Data Size: {len(val_data)}")
print(f"Test Data Size: {len(test_data)}")


100%|██████████| 9.91M/9.91M [00:00<00:00, 14.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 508kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.34MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.97MB/s]


Training Data Size: 48000
Validation Data Size: 12000
Test Data Size: 10000


In [ ]:
# Hyperparameters
batch_size = 128
learning_rate = 0.001
num_epochs = 15
num_qubits = 8  # Quantum circuit size

dev = qml.device("default.qubit", wires=num_qubits)

In [ ]:
# Quantum Circuit
def quantum_layer(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(num_qubits)) #q encoding
    qml.BasicEntanglerLayers(weights, wires=range(num_qubits)) #q gates
    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)] #measurement

weight_shapes = {"weights": (1, num_qubits, num_qubits)}
qnode = qml.QNode(quantum_layer, dev, interface="torch")
quantum_net = qml.qnn.TorchLayer(qnode, weight_shapes)

In [ ]:
# CNN Feature Extractor
class CNNFeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7 * 7 * 32, num_qubits) #first fc layer

    def forward(self, x):
        x = self.conv(x) #CNN layers + Pooling
        x = x.view(x.size(0), -1) # Flatten the output
        x = self.fc(x) #first fully conn layer
        return x

In [ ]:
class HybridQuantumCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = CNNFeatureExtractor()
        self.q_layer = quantum_net
        self.fc2 = nn.Linear(num_qubits, 10)  # Second Fully Connected Layer (10 classes for MNIST)

    def forward(self, x, return_features=False):
        # Extract features from CNN
        cnn_features = self.cnn(x)  # CNN Layers + Pooling + First Fully Connected Layer

        # Pass through quantum layer
        quantum_features = self.q_layer(cnn_features)  # Quantum Encoding + Quantum Gates + Measurement

        # Final classification
        output = self.fc2(quantum_features)  # Second Fully Connected Layer

        if return_features:
            return {
                "cnn_features": cnn_features,  # Features after CNN + First Fully Connected Layer
                "quantum_features": quantum_features,  # Features after Quantum Layer
                "output": output  # Final classification output
            }
        else:
            return output


# *Loss Function & Optimizer and Train*

In [ ]:
# Initialize Model
model = HybridQuantumCNN()

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Optimizer

# Training Function
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        total_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()  # Clear gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
            total_loss += loss.item()

        # Print loss for each epoch
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

# Train the model
train(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

Epoch [1/15], Loss: 1.8342
Epoch [2/15], Loss: 0.8768
Epoch [3/15], Loss: 0.1849
Epoch [4/15], Loss: 0.0873
Epoch [5/15], Loss: 0.0668
Epoch [6/15], Loss: 0.0545
Epoch [7/15], Loss: 0.0452
Epoch [8/15], Loss: 0.0373
Epoch [9/15], Loss: 0.0323
Epoch [10/15], Loss: 0.0286
Epoch [11/15], Loss: 0.0251
Epoch [12/15], Loss: 0.0227
Epoch [13/15], Loss: 0.0176
Epoch [14/15], Loss: 0.0158
Epoch [15/15], Loss: 0.0138


In [ ]:
torch.save(model.state_dict(), "hybrid_model.pth")

# ***Extract Features***

In [ ]:
def extract_features(model, dataloader, layer_names):
    """
    Extract features from multiple layers of the model.

    Args:
        model: The trained model.
        dataloader: DataLoader for the dataset.
        layer_names: List of names of the layers to extract features from.

    Returns:
        features_dict: Dictionary containing extracted features from each layer.
        labels: Corresponding labels.
    """
    model.eval()  # Set the model to evaluation mode
    features_dict = {layer_name: [] for layer_name in layer_names}  # Dictionary to store features
    labels = []  # List to store corresponding labels

    with torch.no_grad():  # Disable gradient computation
        for images, label in dataloader:
            # Forward pass with feature extraction
            outputs = model(images, return_features=True)

            # Extract features from each specified layer
            for layer_name in layer_names:
                layer_features = outputs[layer_name].cpu().numpy()  # Move to CPU and convert to numpy
                features_dict[layer_name].append(layer_features)

            labels.append(label.cpu().numpy())  # Move to CPU and convert to numpy

    # Concatenate all batches for each layer
    for layer_name in layer_names:
        features_dict[layer_name] = np.concatenate(features_dict[layer_name], axis=0)

    labels = np.concatenate(labels, axis=0)

    return features_dict, labels

# # Extract features from the trained model
# features_dict, labels = extract_features(model, train_loader, ["cnn_features", "quantum_features"])

# ***Randomly Select Layers and Extract Features***

In [ ]:
import random

# Initialize and forward pass through the model
model = HybridQuantumCNN()

# List of all layer names in the model
all_layer_names = ["cnn_features", "quantum_features"]  # Add more layer names if needed

# Randomly select a subset of layers
num_layers_to_select = 2  # Number of layers to randomly select
random_layer_names = random.sample(all_layer_names, num_layers_to_select)

# Extract features from the randomly selected layers
features_dict, labels = extract_features(model, train_loader, random_layer_names)

# Print the shapes of the extracted features
for layer_name, features in features_dict.items():
    print(f"Features from layer '{layer_name}' shape: {features.shape}")

Features from layer 'cnn_features' shape: (48000, 8)
Features from layer 'quantum_features' shape: (48000, 8)


In [ ]:
from sklearn.svm import OneClassSVM
from scipy.spatial.distance import hamming

# Train One-Class SVM on classical features
classical_svm = OneClassSVM(gamma='auto', nu=0.1)  # Adjust hyperparameters as needed
classical_svm.fit(features_dict["cnn_features"])

# Train One-Class SVM on quantum features
quantum_svm = OneClassSVM(gamma='auto', nu=0.1)  # Adjust hyperparameters as needed
quantum_svm.fit(features_dict["quantum_features"])

# Get predictions from the classical and quantum SVMs
classical_predictions = classical_svm.predict(features_dict["cnn_features"])
quantum_predictions = quantum_svm.predict(features_dict["quantum_features"])

# Compute Hamming distance between the predictions
hamming_distance = hamming(classical_predictions, quantum_predictions)
print(f"Hamming Distance: {hamming_distance:.4f}")

# Set condition for inliers and outliers
d = 0.1  # Adjust as needed
if hamming_distance < d:
    print("Inlier detected.")
else:
    print("Outlier detected.")

Hamming Distance: 0.0751
Inlier detected.


# ***OOD Detection on New Data***

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from scipy.spatial.distance import hamming

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Verify the dataset path
root = "/content/drive/My Drive/ood"  # Path to the dataset in Google Drive
if not os.path.exists(root):
    raise FileNotFoundError(f"The directory '{root}' does not exist.")

# Verify the subdirectories exist
subdirs = ["cats", "dogs"]
for subdir in subdirs:
    subdir_path = os.path.join(root, subdir)
    if not os.path.exists(subdir_path):
        raise FileNotFoundError(f"The subdirectory '{subdir_path}' does not exist.")

# Define transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize images to 28x28 (same as MNIST)
    transforms.Grayscale(),  # Convert to grayscale (if needed)
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize
])

# Load out-of-distribution data (e.g., cat, dog images)
ood_data = datasets.ImageFolder(root=root, transform=transform)
ood_loader = DataLoader(ood_data, batch_size=batch_size, shuffle=False)

# Extract features from OOD data
features_dict_ood, _ = extract_features(model, ood_loader, ["cnn_features", "quantum_features"])

# Predict using One-Class SVM
classical_predictions_ood = classical_svm.predict(features_dict_ood["cnn_features"])
quantum_predictions_ood = quantum_svm.predict(features_dict_ood["quantum_features"])

# Compute Hamming distance for OOD data
hamming_distance_ood = hamming(classical_predictions_ood, quantum_predictions_ood)
print(f"Hamming Distance for OOD data: {hamming_distance_ood:.4f}")

# Set condition for OOD data
d = 0.1  # Adjust as needed
if hamming_distance_ood < d:
    print("OOD data is classified as inlier.")
else:
    print("OOD data is classified as outlier.")

Hamming Distance for OOD data: 0.1064
OOD data is classified as outlier.


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Combine predictions and labels
all_predictions = np.concatenate([classical_predictions, classical_predictions_ood])
all_labels = np.concatenate([np.ones_like(classical_predictions), -1 * np.ones_like(classical_predictions_ood)])

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, pos_label=1)
recall = recall_score(all_labels, all_predictions, pos_label=1)
f1 = f1_score(all_labels, all_predictions, pos_label=1)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.8997
Precision: 0.9996
Recall: 0.9000
F1-Score: 0.9472
